In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

In [158]:

def resample_cumulative(df):
    """
    """
    df = df.set_index("date", drop = True)
    df = df.resample("10T").last()
    df["value"] = df["value"].diff()*6
    df = df.fillna(0)
    return df

def resample_continuous(df):
    df = df.set_index("date", drop = True)
    return df.resample("10T").mean()


def clean_continuous(filename, name, start, stop):
    df = pd.read_csv(filename, parse_dates= [["date", "time"]]).rename(columns={"date_time":"date"})
    df = resample_continuous(df)
    df = df.rename(columns={"value":name})
    return df[(df.index >= start) &(df.index < stop)]


def clean_cumulative(filename, name, start, stop):
    df = pd.read_csv(filename, parse_dates= [["date", "time"]]).rename(columns={"date_time":"date"})
    df = resample_cumulative(df)
    df = df.rename(columns={"value":name})
    return df[(df.index >= start) &(df.index < stop)]
    
    
    

In [159]:
start = datetime(2021, 3, 9)
stop = datetime(2021, 3, 16)

filenames_cont = ["./air_temp.csv",  "./GHI.csv", "./P1P2_active.csv", "./P2_active.csv", "./SOC.csv", "./wind_speed.csv"]
filenames_cum = ["./P2_load_import.csv", "./elspec_active_import.csv", "./PV_export.csv"]

column_names_cont = ["airTemp", "GHI", "L", "L2_active", "SOC","Wind_speed"]
column_names_cum = ["L2_import", "elspec", "PV"]


air_temp,  GHI, P1P2, P2, SOC, wind_speed = [ clean_continuous(filenames_cont[i], column_names_cont[i], start, stop) for i in range(len(filenames_cont))]
P2_import, elspec, PV = [clean_cumulative(filenames_cum[i], column_names_cum[i], start, stop) for i in range(len(filenames_cum))]
 




In [161]:

df = pd.concat([air_temp, elspec, GHI, P1P2, P2, P2_import, PV, SOC, wind_speed], axis =1)

df.to_csv("../../src/data/09.03_cleaned.csv")